<a href="https://colab.research.google.com/github/GUNWOOOH/ADP_Practice/blob/main/%EA%B8%B0%EC%B6%9C%EB%B3%B5%EC%9B%90/%EA%B8%B0%EC%B6%9C%EB%B3%B5%EC%9B%90_ADP28_%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ADP 28회 기출복원

1. 데이터 차원축소 목적 장단점 과적합 보완방안?(근데 실제로 이거 잘~말고 있나)

tree base 모델이라서 과적합 방지용으로 imbalanced data SMOTE로 over sampling 하는 것이 답에 근사

# 언더샘플링

1. 무작위추출 : 무작위로 정상 데이터를 일부만 선택

2. 유의정보 : 유의한 데이터만을 남기는 방식(알고리즘 : EasyEnsemble, BalanceCascade)

언더샘플링의 경우 데이터의 소실이 매우 크고, 때로는 중요한 정상데이터를 잃게 될 수 있다.


# 오버샘플링

1. 무작위추출 : 무작위로 소수 데이터를 복제

2. 유의정보 : 사전에 기준을 정해서 소수 데이터를 복제

정보가 손실되지 않는다는 장점이 있으나, 복제된 관측치를 원래 데이터 세트에 추가하기 만하면 여러 유형의 관측치를 다수 추가하여 오버 피팅 (overfitting)을 초래할 수 있다.

이러한 경우 trainset의 성능은 높으나 testset의 성능은 나빠질 수 있다.

3. 합성 데이터 생성 : 소수 데이터를 단순 복제하는 것이 아니라 새로운 복제본을 만들어 낸다.

# 비용 민감 학습(Cost Sensitive Learning,CSL)

오분류하는 행위를 비용으로 측정한다.

Total Cost = C(FN)xFN + C(FP)xFP

FN은 잘못 예측 된 긍정적인 관찰의 수

FP는 잘못 예측 된 부정적 사례의 수

C(FN)과 C(FP)는 False Negative 및 False Positive와 관련된 비용과 각각 일치한다. C(FN)> C(FP)

잘못 분류된 비용을 설명하는 비용 매트릭스를 사용하여 불균형 학습 문제를 해결한다.

최근의 이 방법론은 샘플링 기법으로의 대체로 대두되기도 한다.

#SMOTE알고리즘

은 오버샘플링 기법 중 합성데이터를 생성하는 방식으로 가장 많이 사용되고 있는 모델이다.

SMOTE(synthetic minority oversampling technique)란, 합성 소수 샘플링 기술로 다수 클래스를 샘플링하고 기존 소수 샘플을 보간하여 새로운 소수 인스턴스를 합성해낸다.

일반적인 경우 성공적으로 작동하지만, 소수데이터들 사이를 보간하여 작동하기 때문에 모델링셋의 소수데이터들 사이의 특성만을 반영하고 새로운 사례의 데이터 예측엔 취약할 수 있다.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

dataset = load_breast_cancer()
X_features = dataset.data
y_label = dataset.target

scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit_transform(X_features)
X_train = scaler.fit_transform(X_features)

from sklearn.datasets import make_classification
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE

# 모델설정
sm = SMOTE( sampling_strategy='auto')

# train데이터를 넣어 복제함
X_resampled, y_resampled = sm.fit_resample(X_train,list(y_label))

print('After OverSampling, the shape of train_X: {}'.format(X_resampled.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(X_resampled.shape))

print("After OverSampling, counts of label '1': {}".format(sum(y_resampled==1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_resampled==0)))

X_resampled.shape, y_resampled.shape

After OverSampling, the shape of train_X: (714, 30)
After OverSampling, the shape of train_y: (714, 30) 



TypeError: ignored


# 2. LightGBM 랜덤포레스트 neuralnetwork


In [ ]:
#lightgbm

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

mpl.rc('font', family='NanumGothic') # 폰트 설정
mpl.rc('axes', unicode_minus=False) # 유니코드에서 음수 부호 설정

# 차트 스타일 설정
sns.set(font="NanumGothic", rc={"axes.unicode_minus":False}, style='darkgrid')
plt.rc("figure", figsize=(10,8))

warnings.filterwarnings("ignore")

from lightgbm import LGBMClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

dataset = load_breast_cancer()
X_features = dataset.data
y_label = dataset.target

X_train, X_test, y_train, y_test = train_test_split(X_features, y_label, test_size=0.2, random_state=156)

# eval_set
evals = [ (X_test, y_test) ]

# 학습 (조기 중단 지정)
lgbm_wrapper = LGBMClassifier(n_estimators=4000)
lgbm_wrapper.fit(X_train, y_train,
                 # 조기 중단 파라미터
                 early_stopping_rounds = 100, eval_metric = "logloss", eval_set = evals, 
                 verbose=True)

from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.metrics import f1_score, roc_auc_score

def get_clf_eval(y_test, pred=None, pred_proba_po=None):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    auc = roc_auc_score(y_test, pred_proba_po)
   
    print("오차 행렬")
    print(confusion)
    print(f"정확도: {accuracy:.4f}, 정밀도: {precision:.4f}, 재현율: {recall:.4f}, F1: {f1:.4f}, AUC: {auc:.4f}")

preds = lgbm_wrapper.predict(X_test)
pred_proba = lgbm_wrapper.predict_proba(X_test)[:,1]

# 평가
get_clf_eval(y_test, preds, pred_proba)

[1]	valid_0's binary_logloss: 0.565079
[2]	valid_0's binary_logloss: 0.507451
[3]	valid_0's binary_logloss: 0.458489
[4]	valid_0's binary_logloss: 0.417481
[5]	valid_0's binary_logloss: 0.385507
[6]	valid_0's binary_logloss: 0.355773
[7]	valid_0's binary_logloss: 0.329587
[8]	valid_0's binary_logloss: 0.308478
[9]	valid_0's binary_logloss: 0.285395
[10]	valid_0's binary_logloss: 0.267055
[11]	valid_0's binary_logloss: 0.252013
[12]	valid_0's binary_logloss: 0.237018
[13]	valid_0's binary_logloss: 0.224756
[14]	valid_0's binary_logloss: 0.213383
[15]	valid_0's binary_logloss: 0.203058
[16]	valid_0's binary_logloss: 0.194015
[17]	valid_0's binary_logloss: 0.186412
[18]	valid_0's binary_logloss: 0.179108
[19]	valid_0's binary_logloss: 0.174004
[20]	valid_0's binary_logloss: 0.167155
[21]	valid_0's binary_logloss: 0.162494
[22]	valid_0's binary_logloss: 0.156886
[23]	valid_0's binary_logloss: 0.152855
[24]	valid_0's binary_logloss: 0.151113
[25]	valid_0's binary_logloss: 0.148395
[26]	vali


# 3. soft voting Hard voting

출처: https://cori.tistory.com/170

[개요] 

Hard Voting, Soft Voting이 무엇인지 알아보고, 이들을 활용하는 예제를 다뤄본다.

 

[내용 정리]

1. Voting 

0) 정의 

-> 서로 다른 종류의 알고리즘들을 결합하여 다수결 방식으로 최종 결과 출력한다. 

 

1) Hard Voting 

-> 다수의 추정기가 예측한 값들 중 많은 것을 선택한다.

2) Soft Voting

· 서로 다른 종류의 알고리즘들을 결합하여 다수결 방식으로 최종 결과 출력

· 다수의 추정기에서 각 레이블 별 예측한 확률들의 평균을 내서 높은 레이블값을 결과값으로 선택

· Hard Voting보다 Soft Voting의 성능이 더 좋다. 

3) voting 함수 

sklearn.ensemble.VotingClassifier

 

· estimators: 앙상블할 모델들을 설정한다. ("추정기이름", 추정기)의 튜플을 리스트로 묶어서 전달

· voting: voting 방식을 설정한다. (hard: 기본값, soft: 지정) 


# 4-1. 시간별/제조사별 불량률 데이터로 생존분석하여 25, 30, 35개월 후 불량률 계산

lifelines.KaplanMeierFitter

# 4-2. 로그 순위법으로 제조사별 차이 검정

lifelines.statistics.logrank_test

 



# 5. 시식 전 후 구매 의도를 가진 사람 비율 차이 검정 # 교차표 형식으로 표 그림 제공

scipy.stats.chi2_contingency

 



# 6. 2개 열을 가진 성적 데이터로 각 열의 분산 차이 검정 # 각 열의 행 개수가 달랐음

scipy.stats.levene ?

 

7. 몸무게 통제 시 나이와 콜레스테롤 지수 편상관분석 # pingouin.partial_corr



# 7. 몸무게 통제 시 나이와 콜레스테롤 지수 편상관분석 

pingouin.partial_corr